In [ ]:
import math
def Cracking_Moment(): #Calculates cracking moment of a rectangular and T-section

    shape = input("Is it a rectangular or T-section (R or T): ")
    Es = 200000
    fc = float(input("Enter the compressive strength of the concrete in MPa: "))
    Ec = 4500*math.sqrt(fc)
    n = round(Es/Ec)
    print("The modular ratio is", n)

    if shape == "R":
        #Calculating the Virtual Area (Assuming the section is rectangular)
        b = float(input("Enter the width of the concrete section in mm: "))
        h = float(input("Enter the height of the concrete section in mm: "))
        Ac = b*h
        As = float(input("Enter the area of the tension steel in squared mm: "))
        Asc = float(input("Enter the area of the compression steel in squared mm: "))
        Av = Ac + (n-1)*As + (n-1)*Asc
        print("The equivalent virtual area of the section is", Av, "mm^2")
    
        #Calculate centroid of the section (Assuming rectangular)
        dc = float(input("Enter the distance to the tension steel from the bottom of the section in mm: "))
        dprime = float(input("Enter the distance to the compression steel from the bottom of the section in mm: "))
        centroid = (b*h*h/2 + (n-1)*As*dc + (n-1)*Asc*dprime)/Av
        print("The centroid is at", centroid, "mm")
        
        #Calculate MOI of the section (Assuming rectangular section)
        Ig = (b*h**3)/12 + b*h*(h/2 - centroid)**2 + (n-1)*As*(centroid - dc)**2 + (n-1)*Asc*(dprime-centroid)**2
        print("The moment of interia to of the virtual section is", Ig, "mm^4")
        
    elif shape == "T":
        
        #Calculating the Virtual Area (Assuming the section is T-shaped)
        flange_width = float(input("Enter the width of the flange in mm: "))
        flange_height = float(input("Enter the height of the flange section in mm: "))
        web_width = float(input("Enter the width of the web of the section in mm: "))
        web_height = float(input("Enter the height of the web of the section in mm: "))
        Ac = flange_width*flange_height + web_width*web_height
        As = float(input("Enter the area of the tension steel in squared mm: "))
        Av = Ac + (n-1)*As
        
        #Calculate centroid of the section (Assuming T-section)
        dc = float(input("Enter the distance to the tension steel from the bottom of the section in mm: "))
        centroid = (flange_width*flange_height*(web_height+flange_height/2) + web_width*web_height*(web_height/2) + (n-1)*As*dc)/Av
        
        #Calculate MOI of the section (Assuming T-shaped section)
        Ig = (flange_width*flange_height**3)/12 + flange_width*flange_height*(web_height+flange_height/2 - centroid)**2 + (web_width*web_height**3)/12 + web_width*web_height*(web_height/2 - centroid)**2 +(n-1)*As*(centroid-dc)**2
        
    #Calculate concrete modulus of rupture
    fr = 0.6*math.sqrt(fc)
    
    #Caluclate the cracking moment
    Mcr = (fr*Ig)/centroid
    print("The cracking moment of the section is", Mcr, "N*mm")
    print("The cracking moment of the section is", Mcr/10**6, "kN*m")
    
    return Mcr

In [ ]:
from sympy import symbols,Eq, solve
def Moment_Resistance(): #For Rectangular sections only (Does not take compression steel into account)
    Es = 200000
    fc = float(input("Enter the compressive strength of the concrete in MPa: "))
    fy = float(input("Enter the yield stress of the steel in MPa: "))
    phi_c = 0.65
    phi_s = 0.85
    strain_conc = 0.0035
    steel_yield_strain = fy/Es
    alpha1 = 0.85 - 0.0015*fc
    beta1 = 0.97 - 0.0025*fc
    
    b = float(input("Enter the width of the concrete section in mm: "))
    h = float(input("Enter the height of the concrete section in mm: "))
    As = float(input("Enter the area of the tension steel in squared mm: "))
    dcover = float(input("Enter the distance to the tension steel from the bottom of the section in mm: "))
    d = h-dcover
    
    a = (phi_s*fy*As)/(b*fc*phi_c*alpha1)
    C = a/beta1
    
    strain_tension_steel = ((d - C)/C)*strain_conc
    
    if strain_tension_steel >= steel_yield_strain: #Calculating Mr if the previous assumptions are correct
        Mr = alpha1*phi_c*fc*a*b*(d-a/2)
        print("The moment resistance of the section is", Mr, "N*mm")
        print("The moment resistance of the section is", Mr/10**6, "kN*m")
        print("The concrete section is under-reinforced")
    else:            #if the assumptions are incorrect
        C, fs = symbols('C fs')
        
        eq1 = Eq((alpha1*phi_c*fc*beta1*C*b)/(phi_s*fs*As),1)
        
        eq2 = Eq(fs*C/700+C-d,0)
        
#solving the system of two eqns and two unknowns 
        solution = solve((eq1,eq2), (C, fs))
        
#turning the tuples of solutions into list of solutions        
        list_solution = []
        
        for i in range(len(solution)):
            list_solution.append(list(solution[i]))
            
#making a list for the positive solutions from the existing list of solutions         
        posi_solution = []
        for m in range(len(list_solution)):
            for n in range(len(list_solution)):
                if list_solution[m][n]>0:
                    posi_solution.append(list_solution[m][n])
        
        C = posi_solution[0]
        fs = posi_solution[1]
        
#Calculating Moment resistance in the case of wrong assumptions
        Mr = alpha1*phi_c*fc*beta1*C*b*(d-beta1*C*0.5)
        
        print("The moment resistance of the section is", Mr, "N*mm")
        print("The moment resistance of the section is", Mr/10**6, "kN*m")
        print("The concrete section is over-reinforced")
        
    return Mr

In [ ]:
from sympy import symbols,Eq, solve
def Moment_Resistance1(): #Calculates moment resistance for Rectangular sections only (takes compression and tension steel into account)
    Es = 200000
    fc = float(input("Enter the compressive strength of the concrete in MPa: "))
    fy = float(input("Enter the yield stress of the steel in MPa: "))
    phi_c = 0.65
    phi_s = 0.85
    strain_conc = 0.0035
    steel_yield_strain = fy/Es
    alpha1 = 0.85 - 0.0015*fc
    beta1 = 0.97 - 0.0025*fc
    
    b = float(input("Enter the width of the concrete section in mm: "))
    h = float(input("Enter the height of the concrete section in mm: "))
    As = float(input("Enter the area of the tension steel in squared mm: "))
    dcover = float(input("Enter the distance to the tension steel from the bottom of the section in mm: "))
    Asc = float(input("Enter the area of the compression steel in squared mm: "))
    dprime = float(input("Enter the distance to the compression steel from the top of the section in mm: "))
    d = h-dcover
    
    a = (phi_s*fy*As-phi_s*fy*Asc)/(b*fc*phi_c*alpha1)
    C = a/beta1
    
    strain_tension_steel = ((d - C)/C)*strain_conc
    strain_comp_steel = ((C - dprime)/C)*strain_conc
    
    if strain_tension_steel >= steel_yield_strain and strain_comp_steel >= steel_yield_strain: #Calculating Mr if the previous assumptions are correct
        Mr = alpha1*phi_c*fc*a*b*(d-a/2) + phi_s*fy*Asc*(d - dprime)
        print("The moment resistance of the section is", Mr, "N*mm")
        print("The moment resistance of the section is", Mr/10**6, "kN*m")
        print("The concrete section is under-reinforced and both assumptions are correct")
    
    elif strain_tension_steel >= steel_yield_strain and strain_comp_steel < steel_yield_strain:    #if the second assumption is incorrect
        C, fsprime = symbols('C fsprime')
        
        eq1 = Eq((alpha1*phi_c*fc*beta1*C*b + phi_s*fsprime*Asc)/(phi_s*fy*As),1)
        
        eq2 = Eq(fsprime*C/700- C+dprime,0)
        
#solving the system of two eqns and two unknowns 
        solution = solve((eq1,eq2), (C, fsprime))
    
#turning the tuples of solutions into list of solutions        
        list_solution = []
        
        for i in range(len(solution)):
            list_solution.append(list(solution[i]))
            
        solution1 = list_solution[0]
        solution2 = list_solution[1]
        
        if solution1[0]>0 and solution1[0]<h:
            C = solution1[0]
        else:
            C = solution2[0]
            
        if solution1[1]>0 and solution1[1]<fy:
            fsprime = solution1[1]
        else:
            fsprime = solution2[1]        
        
        a = beta1*C
#Calculating Moment resistance in the case of wrong assumptions
        Mr = alpha1*phi_c*fc*a*b*(d-a/2)+phi_s*fsprime*Asc*(d-dprime)
    
        print("The moment resistance of the section is", Mr, "N*mm")
    
        print("The moment resistance of the section is", Mr/10**6, "kN*m")
        
        print("The second assumption is incorrect and only the tension steel has yield")
        
    elif strain_tension_steel < steel_yifeld_strain and strain_comp_steel >= steel_yield_strain:
        C, fs = symbols('C fs')
        
        eq1 = Eq((alpha1*phi_c*fc*beta1*C*b + phi_s*fy*Asc)/(phi_s*fs*As),1)
        
        eq2 = Eq(fs*C/700+C-d,0)
        
        #solving the system of two eqns and two unknowns 
        solution = solve((eq1,eq2), (C, fs))
    
#turning the tuples of solutions into list of solutions        
        list_solution = []
        
        for i in range(len(solution)):
            list_solution.append(list(solution[i]))
            
        solution1 = list_solution[0]
        solution2 = list_solution[1]
        
        if solution1[0]>0 and solution1[0]<h:
            C = solution1[0]
        else:
            C = solution2[0]
            
        if solution1[1]>0 and solution1[1]<fy:
            fs = solution1[1]
        else:
            fs = solution2[1]        
        
        a = beta1*C
        
        Mr = alpha1*phi_c*fc*a*b*(d-a/2)+phi_s*fy*Asc*(d-dprime)
        
        print("The moment resistance of the section is", Mr, "N*mm")
    
        print("The moment resistance of the section is", Mr/10**6, "kN*m")
        
        print("The first assumption is incorrect and only the compression steel has yield")
    return Mr

In [ ]:
from sympy import symbols,Eq, solve
def Moment_ResistanceT(): #Calculates the moment resistance of a T-beam section in flexure
    Es = 200000
    fc = float(input("Enter the compressive strength of the concrete in MPa: "))
    fy = float(input("Enter the yield stress of the steel in MPa: "))
    phi_c = 0.65
    phi_s = 0.85
    strain_conc = 0.0035
    steel_yield_strain = fy/Es
    alpha1 = 0.85 - 0.0015*fc
    beta1 = 0.97 - 0.0025*fc
    
    bf = float(input("Enter the width of the entire flange in mm: "))
    hs = float(input("Enter the height of the flange in mm: "))
    bw = float(input("Enter the width of the entire web in mm: "))
    h = float(input("Enter the total height of the section in mm: "))
    As = float(input("Enter the area of the tension steel in squared mm: "))
    dc = float(input("Enter the distance to the tension steel from the bottom of the section in mm: "))
    d = h - dc
    
    a = (phi_s*fy*As)/(alpha1*phi_c*fc*bf)
    C = a/beta1
    
    strain_tension_steel = (strain_conc/C)*(d-C)
    
    if a <= hs and strain_tension_steel >= steel_yield_strain:
        Mr = alpha1*phi_c*fc*a*bf*(d-a/2)
        
        print("The moment resistance of the section is", Mr, "N*mm")
        print("The moment resistance of the section is", Mr/10**6, "kN*m")
        print("The concrete section is under-reinforced and both assumptions are correct")    
    
    elif a> hs:
        
        new_a = (phi_s*fy*As-alpha1*phi_c*fc*(bf-bw)*hs)/(alpha1*phi_c*fc*bw)

        C = new_a/beta1
        print (new_a,C)
        strain_tension_steel = (strain_conc/C)*(d-C)
        
        if strain_tension_steel >= steel_yield_strain:
            Mr = alpha1*phi_c*fc*(C*beta1)*bw*(d-a/2) + alpha1*phi_c*fc*(bf-bw)*hs*(d - hs/2)
            print("The moment resistance of the section is", Mr, "N*mm")
    
            print("The moment resistance of the section is", Mr/10**6, "kN*m")
        
            print("The first assumption was incorrect")
        
        elif strain_tension_steel < steel_yield_strain:
            C, fs = symbols('C fs')
        
            eq1 = Eq((alpha1*phi_c*fc*beta1*C*bw + alpha1*phi_c*fc*(bf-bw)*hs)/(phi_s*fs*As),1)
        
            eq2 = Eq(fs*C/700+C-d,0)
        
            #solving the system of two eqns and two unknowns 
            solution = solve((eq1,eq2), (C, fs))
            
            #turning the tuples of solutions into list of solutions        
            list_solution = []
        
            for i in range(len(solution)):
                list_solution.append(list(solution[i]))
            
            solution1 = list_solution[0]
            solution2 = list_solution[1]
            
            if solution1[0]>0 and solution1[0]<h:
                C = solution1[0]
            
            else:
                C = solution2[0]
            
            if solution1[1]>0 and solution1[1]<fy:
                fs = solution1[1]
            else:
                fs = solution2[1]        
        
            a = beta1*C
        
            Mr = alpha1*phi_c*fc*a*bw*(d-a/2) + alpha1*phi_c*fc*(bf-bw)*hs*(d-hs/2)
            
            print("The moment resistance of the section is", Mr, "N*mm")
    
            print("The moment resistance of the section is", Mr/10**6, "kN*m")
        
            print("Both assumptions were incorrect. The reinforcement did not yield")
            
    return Mr

In [ ]:
import math 
def steelratio(fy,fc): #Calculates the maximum and minimum steel ratios
    phi_c = 0.65
    phi_s = 0.85 
    strain_conc = 0.0035
    alpha1 = 0.85 - 0.0015*fc
    beta1 = 0.97 - 0.0025*fc
    rho_max = 0.8*alpha1*beta1*(phi_c/phi_s)*(fc/fy)*(700/(700+fy))
    rho_min = 0.2*math.sqrt(fc)/fy
    print("The maximum steel ratio is ",rho_max*100, "% and The minimum steel ratio is ", rho_min*100,"%")
    return [rho_max , rho_min]

In [ ]:
def beamhmin(l): #Calculates the minimum height required of a beam cross-section 
    beam = input("Enter the beam type; simply supported(A), Cantilever(B), One end continuous (C), both ends continuous(D): ")
    if beam == "A":
        beamhmin = l/16
        print ("The minimum height of the cross-section is", beamhmin, "mm")
    elif beam == "B":
        beamhmin = l/8
        print ("The minimum height of the cross-section is", beamhmin, "mm")
    elif beam == "C":
        beamhmin = l/18
        print ("The minimum height of the cross-section is", beamhmin, "mm")
    elif beam == "D":
        beamhmin = l/21
        print ("The minimum height of the cross-section is", beamhmin, "mm")
    return beamhmin

In [ ]:
def compressionzone(fy,d): #Calculates Cmax for a section
    Cbalance = (700/(700+fy))*d
    Cmax = 0.8*Cbalance
    print("Cmax is", Cmax,"mm")
    return Cmax

In [ ]:
def beamflexuredesign1(fc,fy,fyprime,b): #Calculates the area of the tension and compression steel for a rectangular RC section subject to flexure 
    h = float(input("Enter an appropriate height for the cross-section in mm: ")) #Enter a height greater than hmin (check with hmin function above)
    d = h - 50
    Mf = float(input("Enter the factored moment around the tension steel in kN*m: "))*10**6
    phi_c = 0.65
    phi_s = 0.85
    alpha1 = 0.85 - 0.0015*fc
    beta1 = 0.97 - 0.0025*fc
    
    # Solve the quadratic equation mx**2 + nx + p = 0
    import math

    m = -1/2
    n = d
    p = -Mf/(alpha1*phi_c*fc*b)
    
    # calculate the discriminant
    r = (n**2) - (4*m*p)

    if r < 0:
        print ("This equation has no real solution")
    
    elif r == 0:
        a = (-n+math.sqrt(n**2-4*m*p))/(2*m)
    
        C = a/beta1
    
        Cmax = compressionzone(fy,d)
        print ("C= ",C,"mm")
        print("a = ", a, "mm")
    
        if C <= Cmax:
            As = (alpha1*phi_c*fc*a*b)/(phi_s*fy)
            Asprime = 0
            print("The area of tension steel needed is", As, "mm^2")
        elif input("Will compression steel be added? (Y/N) ") == Y:
            a = beta1*Cmax
            dprime = float(input("Enter the distance to the compression steel from the top: "))
            Asprime = (Mf - alpha1*phi_c*fc*a*b*(d-a/2))/(phi_s*fy*(d-dprime))
            As = (alpha1*phi_c*fc*a*b+phi_s*fy*Asprime)/(phi_s*fy)
            
            print("The compression and tension steel are ", Asprime, "mm^2 and", As, "mm^2, respectively")
        else:
            print("Height of the section must be increased")
             
    elif r > 0:
        a1 = (-n+math.sqrt(n**2-4*m*p))/(2*m)
        a2 = (-n-math.sqrt(n**2-4*m*p))/(2*m)
    
        Cmax = compressionzone(fy,d)
        
        if a1 > 0 and a1 < Cmax:
            a = a1
        else:
            a = a2
            
        C = a/beta1
        print ("C= ",C,"mm")
        print("a = ", a, "mm")
        
        if C <= Cmax:
            As = (alpha1*phi_c*fc*a*b)/(phi_s*fy)
            Asprime = 0
            print("The area of tension steel needed is", As, "mm^2 :D")
        
        elif input("Will compression steel be added? (Y/N) ") == "Y":
            a = beta1*Cmax
            dprime = float(input("Enter the distance to the compression steel from the top: "))
            Asprime = (Mf - alpha1*phi_c*fc*a*b*(d-a/2))/(phi_s*fy*(d-dprime))
            As = (alpha1*phi_c*fc*a*b+phi_s*fy*Asprime)/(phi_s*fy)
            
            print("The compression and tension steel are ", Asprime, "mm^2 and", As, "mm^2, respectively")
        else:
            print("Height of the section must be increased")
    return [As,Asprime]

In [ ]:
import math
def min_steel(fc,fy): #Calculates the minimum steel required for a given RC cross-section
    bt = float(input("Enter the dimension for the base of the rectangular portion of the concrete section in mm: "))
    h = float(input("Enter the height of the section in mm: "))
    Asmin = 0.2*math.sqrt(fc)*bt*h*(1/fy)
    print("The minimum steel required for the section is", Asmin,"mm^2")
    return Asmin

In [ ]:
def Tbeamflexuredesign(fc,fy,bf,bw): #Calculates the required steel for a T-beam subject to flexure, requires fc, fy, bf, Mf and bw as inputs
    h = float(input("Enter an appropriate height for the entire cross-section in mm: ")) #Enter a height greater than hmin (check with hmin function above)
    hs = float(input("Enter the height of the flange in mm: "))
    d = h - 50
    Mf = float(input("Enter the factored moment around the tension steel in kN*m: "))*10**6
    phi_c = 0.65
    phi_s = 0.85
    alpha1 = 0.85 - 0.0015*fc
    beta1 = 0.97 - 0.0025*fc
    import math        
    
    Mr_flange = alpha1*phi_c*fc*hs*bf*(d-hs/2)
    print(Mr_flange)
    
    if Mr_flange >= Mf:
        print("a is less than hs")
        
        # Solve the quadratic equation mx**2 + nx + p = 0

        m = -1/2
        n = d
        p = -Mf/(alpha1*phi_c*fc*bf)
    
    # calculate the discriminant
        r = (n**2) - (4*m*p)

        if r < 0:
            print ("This equation has no real solution")
    
        elif r == 0:
            a = (-n+math.sqrt(n**2-4*m*p))/(2*m)
            
            C = a/beta1
            
            Cmax = compressionzone(fy,d)
            print ("C= ",C,"mm")
            print("a = ", a, "mm")
            
            if C <= Cmax:
                As = (alpha1*phi_c*fc*a*bf)/(phi_s*fy)
                print("The area of tension steel is ", As, "mm^2")
            else:
                print("Increase the height of the secion")
        elif r > 0:
            a1 = (-n+math.sqrt(n**2-4*m*p))/(2*m)
            a2 = (-n-math.sqrt(n**2-4*m*p))/(2*m)
    
            Cmax = compressionzone(fy,d)
        
            if a1 > 0 and a1 < Cmax:
                a = a1
            else:
                a = a2
            
            C = a/beta1
            print ("C= ",C,"mm")
            print("a = ", a, "mm")
            
            if C <= Cmax:
                As = (alpha1*phi_c*fc*a*bf)/(phi_s*fy)
                print("The area of tension steel is ", As, "mm^2")
            else:
                print("Increase the height of the secion")
    else:
        print("a is greater than hs")
        
                # Solve the quadratic equation mx**2 + nx + p = 0

        m = -1/2
        n = d
        p = -(Mf-alpha1*phi_c*fc*hs*(bf-bw)*(d-hs/2))/(alpha1*phi_c*fc*bw)
    
    # calculate the discriminant
        r = (n**2) - (4*m*p)

        if r < 0:
            print ("This equation has no real solution")
    
        elif r == 0:
            a = (-n+math.sqrt(n**2-4*m*p))/(2*m)
            
            C = a/beta1
            
            Cmax = compressionzone(fy,d)
            print ("C= ",C,"mm")
            print("a = ", a, "mm")
            
            if C <= Cmax:
                As = (alpha1*phi_c*fc*a*bw+alpha1*phi_c*fc*hs*(bf-bw))/(phi_s*fy)
                print("The area of tension steel is ", As, "mm^2")
            else:
                print("Increase the height of the secion")
        elif r > 0:
            a1 = (-n+math.sqrt(n**2-4*m*p))/(2*m)
            a2 = (-n-math.sqrt(n**2-4*m*p))/(2*m)
    
            Cmax = compressionzone(fy,d)
        
            if a1 > 0 and a1 < Cmax:
                a = a1
            else:
                a = a2
            
            C = a/beta1
            print ("C= ",C,"mm")
            print("a = ", a, "mm")
            
            if C <= Cmax:
                As = (alpha1*phi_c*fc*a*bw+alpha1*phi_c*fc*hs*(bf-bw))/(phi_s*fy)
                print("The area of tension steel is ", As, "mm^2")
            else:
                print("Increase the height of the secion")
    return As
        